<h1><center> Binary Classifier Performance Dashboard</center></h1>


[Binary Classifiers](https://en.wikipedia.org/wiki/Binary_classification) are widely used for various tasks in ML & Data Science. Regardless of the choice of ML algorithms (Logistic, Tree based or Neural Nets), a binary classifiers performance is often optimized or measured based on a select few metrics like AUC-ROC, F-score, etc. 

This App is an attempt to allow ML/DS practitioners more time to understand their results by the way of
- Automating the process of creating some of the popular metrics
- Creating an informative visual dashboard to aid optimal threshold selection

**Metrics of interest in this App**
- AUC-ROC curve & AUC
- N-tile (x-axis) V Target rate (Y-axis) charts - useful viz for how well your model slopes target
- Best possible F-scores and the corresponding threshold values and confusion matrices
- For a threshold of users choosing F-scores and the confusion matrix


    
<h3><center>Directions For Use:</center></h3>   

    
1. Upload a csv with two columns- target variable and model predictions
2. You will see an AUC-ROC curve & the AUC measure
3. Select the number of buckets(n-tile) using the slider for the N-tile V Target rate chart
4. Row 2 conatins the best possible F-scores and corresponding thresholds, confusion matrices. 
5. Adjust the slider to a threshold of your choosing to create the corresponding metrics - precision, recall and confusion matrix.

$\color{#f03c15}{**Note**}$: Column names must be target and preds, the target variable must be dichotomous only taking the values 0 and 1



In [1]:
## import the req libraries
import numpy as np
import pandas as pd
import ipywidgets as widgets
from PIL import Image
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, recall_score, precision_score
import io
from ipywidgets import VBox, HBox, Layout

## graphing tools
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=2)
from  matplotlib.ticker import PercentFormatter

In [2]:
##upload button - for the dataset containing target and model prediction
btn_upload = widgets.FileUpload(accept = 'text/csv'
                                ,align_items='center')

In [3]:
## slider for Threshold
sldr = widgets.FloatSlider(value=0.500
                           , min=0.000
                           , max=1.000
                           , step=0.001
                           , description='Threshold:'
                           , disabled=False
                           , continuous_update=True
                           , orientation='horizontal'
                           , align_items='center'
                           , readout=True
                           , readout_format='.3f')
## slider for n-tiles
ntile_sldr = widgets.IntSlider(value=10
                               , min=5
                               , max=25
                               , step=5
                               , description='N-tiles:'
                               , disabled=False
                               , continuous_update=True
                               , orientation='horizontal'
                               , align_items='center'
                               , readout=True
                               , readout_format='d')

In [4]:
out = widgets.Output()
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred1 = widgets.Label()
lbl_pred2 = widgets.Label()
lbl_predPoint5 = widgets.Label()
lbl_pred1Tr = widgets.Label()
lbl_pred2Tr = widgets.Label()
lbl_predPoint5Tr = widgets.Label()
lbl_user_precision = widgets.Label()
lbl_user_recall = widgets.Label()

In [5]:
## helper func for creating n-tiles
def true_ntile_dist(x_data, ntile = 10):
    
    ## model scores on the dataset you want to build the ntiles off of
    score = x_data['preds']
    
    ## ntile calculation
    ntile_list = [0]
    nvar = int(100/ntile)
    for i in range(1,ntile+1):
        a = (np.percentile(score, nvar*i))
        ntile_list.insert(len(ntile_list),a)
       
    ntile_list[-1] = 1
    
    ## creating the empty dataframe
    df_ntiled = x_data
    try :
        df_ntiled['N-tile buckets'] = pd.cut(score, ntile_list, labels = [i for i in range(1,ntile+1)])
        df = df_ntiled[['N-tile buckets','target']].groupby('N-tile buckets').mean().reset_index()
        df['Target Rate'] = df['target']
    except ValueError:
        print('Error: Probably exceeded the number of bins possible on your data')
        
    ## return the deciled dataframe and the decile cuts     
    return df, ntile_list

In [6]:
## the onclick event for upload button
def on_click(change):
    ## read in the upload as pandas dataframe
    tabl = pd.read_csv(io.BytesIO(btn_upload.data[-1]))
    
    ## clear output
    out.clear_output()
    out1.clear_output()
    out2.clear_output()
    out3.clear_output()

    ## calculating the auc
    auc_ = round(roc_auc_score(tabl['target'], tabl['preds']),3)
    
    ## graphing auc roc curve
    fpr, tpr, _ = roc_curve(tabl['target'],  tabl['preds'])
    fig = plt.figure(figsize=(10,6))
    plt.plot(fpr, 
             tpr, 
             label="Model AUC={:.3f}".format(auc_))
    
    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("Flase Positive Rate", fontsize=15)

    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)
    
    plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='lower right')
    
    with out: plt.show()
    lbl_pred.value = f'AUC of the model {auc_}'
    
    ## calculating F-scores and corresponding threshold
    precision, recall, thresholds = precision_recall_curve(tabl['target']
                                                           , tabl['preds'])
    ## F-1
    f1_scores = 2*recall*precision/(recall+precision)
    best_f1 = round(np.max(f1_scores),3)
    thr_f1 = round(thresholds[np.argmax(f1_scores)],3)
    cnfus1 = pd.crosstab(tabl['target']
                         , (tabl['preds']>=thr_f1).astype(int)
                         , rownames = ['Target']
                         , colnames = ['Predicted'])
    
    sns.heatmap(cnfus1, annot=True
                , cbar = False, fmt = 'd'
                , annot_kws={"size": 24})
    with out1: plt.show()
    lbl_pred1.value = f'Best F1 Score: {best_f1}'
    lbl_pred1Tr.value = f'@ Threshold: {thr_f1}'

    
    ## F-2
    f2_scores = 5*recall*precision/(recall+(4*precision))
    best_f2 = round(np.max(f2_scores),3)
    thr_f2 = round(thresholds[np.argmax(f2_scores)],3)
    cnfus2 = pd.crosstab(tabl['target']
                         , (tabl['preds']>=thr_f2).astype(int)
                         , rownames = ['Target']
                         , colnames = ['Predicted'])
    
    sns.heatmap(cnfus2, annot=True
                , cbar = False, fmt = 'd'
                , annot_kws={"size": 24})
    with out2: plt.show()
    lbl_pred2.value = f'Best F2 Score: {best_f2}'
    lbl_pred2Tr.value = f'@ Threshold: {thr_f2}'

    
    ## F-0.5
    fPoint5_scores = 1.25*recall*precision/(recall+(0.25*precision))
    best_fPoint5 = round(np.max(fPoint5_scores),3)
    thr_fPoint5 = round(thresholds[np.argmax(fPoint5_scores)],3)
    cnfusPoint5 = pd.crosstab(tabl['target']
                              , (tabl['preds']>=thr_fPoint5).astype(int)
                              , rownames = ['Target']
                              , colnames = ['Predicted'])
    
    sns.heatmap(cnfusPoint5, annot=True
                , cbar = False, fmt = 'd'
                , annot_kws={"size": 24})
    with out3: plt.show()
    lbl_predPoint5.value = f'Best F0.5 Score: {best_fPoint5}'
    lbl_predPoint5Tr.value = f'@ Threshold: {thr_fPoint5}'

def chart(change):
    tabl = pd.read_csv(io.BytesIO(btn_upload.data[-1]))
    
    out5.clear_output()
    
    ## ntile V target rate chart 
    x, y = true_ntile_dist(tabl, ntile_sldr.value)
    fig = plt.figure(figsize=(10,6))
    g = sns.barplot(x = 'N-tile buckets'
                    , y = 'Target Rate'
                    , data = x)

    g.axes.yaxis.set_major_formatter(PercentFormatter(1)) 
    # Show the plot
    with out5: plt.show()
        
def response(change):
    
    tabl = pd.read_csv(io.BytesIO(btn_upload.data[-1]))
    
    out4.clear_output()

    ## user defined threshold
    preds_user_thresh = (tabl['preds']>=sldr.value).astype(int)
    precision_user = round(precision_score(tabl['target']
                                           , preds_user_thresh
                                           , average = 'binary'),3)
    recall_user = round(recall_score(tabl['target']
                                     , preds_user_thresh
                                     , average = 'binary'),3)
    cnfususer = pd.crosstab(tabl['target']
                            , preds_user_thresh
                            , rownames = ['Target']
                            , colnames = ['Predicted'])
    lbl_user_precision.value = f'Precision: {precision_user}'
    lbl_user_recall.value = f'Recall: {recall_user}'
    sns.heatmap(cnfususer, annot=True
                , cbar = False, fmt = 'd'
                , annot_kws={"size": 24})
    with out4: plt.show()

In [7]:
btn_upload.observe(on_click, names=['data'])
sldr.observe(response, names ='value')
ntile_sldr.observe(chart, names ='value')

In [8]:
left_box = VBox([lbl_pred1
                 , lbl_pred1Tr
                 , out1])
left_box.layout.align_items = 'center'
    
centr_box = VBox([lbl_pred2
                  , lbl_pred2Tr
                  , out2])
centr_box.layout.align_items = 'center'
   
right_box = VBox([lbl_predPoint5
                  , lbl_predPoint5Tr
                  , out3])
right_box.layout.align_items = 'center'

row1_right_box = VBox([out,lbl_pred]) 
row1_right_box.layout.align_items = 'center'

row1_left_box = VBox([out5,ntile_sldr])
row1_left_box.layout.align_items = 'center'

In [9]:
##organize widgets
vb = VBox([widgets.Label('Select your dataset')
           , btn_upload
           , HBox([row1_right_box,row1_left_box])] 
           , layout=Layout(border='solid'
                           , width='80%'
                           , margin='auto'))

vb3 = VBox([HBox([left_box, centr_box, right_box])]
                  ,layout=Layout(border='solid'
                                 , width='80%'
                                 , margin='auto'))
           
vb.layout.align_items = 'center'
vb3.layout.align_items = 'center'
display(vb)
display(vb3)

In [10]:
vb2 = VBox([widgets.Label('Select the prediction Threshold')
            , sldr 
            , lbl_user_precision
            , lbl_user_recall
            , out4])
vb2.layout.align_items = 'center'
display(vb2)

#### Notes

Precision = $\frac{TP}{TP + FP}$	

Recall = $\frac{TP}{TP + FN}$

F-Score = 2 . $\frac{precision . recall}{precision + recall}$

F $\beta$-Score = (1+$\beta^{2}$) . $\frac{precision . recall}{ \beta^{2}.precision + recall}$

Author - **Prabhat R.**